|**Name**| **ID #** | **Email** | **When Midterm**| 
| :--     |  :--      |  :--    | :--             |           
|Xuan Xu |   111675940 | xuaxu@cs.stonybrook.edu        |                 |             
|Hae-Na Lee| 111207004 | haenalee@cs.stonybrook.edu | 10/13, 10/23 |            
|Caitao Zhan| 111634527| caitao.zhan@stonybrook.edu | 10/16 (CSE548) |



In [1]:
print("Hello Homework3!")
print("Always remember to git pull before you git push.")
print("When a conflict occurs, don't be nervous. Just manually fix the conflicts.")
print("Also, you can .gitignore whatever you don't want others see.")
print("Hope we can finish task 1,2,3 by Sunday night (10/8).")

Hello Homework3!
Always remember to git pull before you git push.
When a conflict occurs, don't be nervous. Just manually fix the conflicts.
Also, you can .gitignore whatever you don't want others see.
Hope we can finish task 1,2,3 by Sunday night (10/8).


In [2]:
# import all packages needed

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, time
from sklearn import linear_model

In [3]:
# To all: let's all use the following names:
# 'df_house16', 'df_house17', 'df_train16', 'df_train17', 'df_merge16', 'df_merge17'

df_house16 = pd.read_csv('../data/properties_2016.csv', na_values=['NA'])
df_train16 = pd.read_csv('../data/train_2016_v2.csv', na_values=['NA'])
# df_merge16 = pd.merge(df_house16, df_train16, on='parcelid')

df_house17 = pd.read_csv('../data/properties_2017.csv', na_values=['NA'])
df_train17 = pd.read_csv('../data/train_2017.csv', na_values=['NA'])
#df_merge17 = pd.merge(df_house17, df_train17, on='parcelid')

/Users/haenalee/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/haenalee/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Task 1

In [4]:
# Read the property data
property_data_2016 = df_house16
property_data_2017 = df_house17

In [5]:
# Extract the features to analyze
feature_list = ['parcelid','bedroomcnt','buildingqualitytypeid',
                'calculatedbathnbr','calculatedfinishedsquarefeet',
                'latitude','longitude','lotsizesquarefeet',
                'yearbuilt','structuretaxvaluedollarcnt']

feature_data_2016 = property_data_2016[feature_list]
feature_data_2017 = property_data_2017[feature_list]

In [6]:
# Read the training data
train_2016 = df_train16
train_2017 = df_train17

In [7]:
# Combine the feature data and training data
model_dataset_2016 = pd.merge(feature_data_2016, train_2016, on='parcelid')
model_dataset_2017 = pd.merge(feature_data_2017, train_2017, on='parcelid')

# Fill in missing feature data with mean values
model_dataset_filled_2016 = model_dataset_2016.fillna(model_dataset_2016.mean())
model_dataset_filled_2017 = model_dataset_2017.fillna(model_dataset_2017.mean())

In [8]:
model_dataset = [model_dataset_filled_2016, model_dataset_filled_2017]
model_dataset = pd.concat(model_dataset, ignore_index=True)

In [9]:
# Preprocessing
model_dataset_norm = model_dataset

# Normalize the features with skewed distributions
model_dataset_norm.calculatedbathnbr = np.log(model_dataset_norm.calculatedbathnbr)
model_dataset_norm.calculatedfinishedsquarefeet = np.log(model_dataset_norm.calculatedfinishedsquarefeet)
model_dataset_norm.lotsizesquarefeet = np.log(model_dataset_norm.lotsizesquarefeet)
#model_dataset_norm.roomcnt = np.log(model_dataset_norm.roomcnt)
model_dataset_norm.structuretaxvaluedollarcnt = np.log(model_dataset_norm.structuretaxvaluedollarcnt)

In [10]:
from sklearn import preprocessing

model_dataset_norm.loc[:,'bedroomcnt':'structuretaxvaluedollarcnt'] = \
 preprocessing.scale(model_dataset_norm.loc[:,'bedroomcnt':'structuretaxvaluedollarcnt'])

In [11]:
feature_list = ['parcelid','bedroomcnt','buildingqualitytypeid',
                'calculatedbathnbr','calculatedfinishedsquarefeet',
                'lotsizesquarefeet',#'roomcnt',
                'yearbuilt','structuretaxvaluedollarcnt','transactiondate']

scoring_dataset = model_dataset_norm[feature_list]

desirability_score = pd.DataFrame(scoring_dataset.bedroomcnt
 - scoring_dataset.buildingqualitytypeid
 + scoring_dataset.calculatedbathnbr
 + scoring_dataset.calculatedfinishedsquarefeet
 + scoring_dataset.lotsizesquarefeet
 + scoring_dataset.yearbuilt
 + scoring_dataset.structuretaxvaluedollarcnt, columns=['score'])

In [12]:
# Append the desirability score to the dataset
scoring_result = pd.concat([scoring_dataset, desirability_score], axis=1)

In [13]:
# Sort based on scores
scoring_result_ordered = scoring_result.sort_values('score', ascending=False)

In [14]:
# Get the ten most desirable houses in the dataset
most_desirables = scoring_result_ordered[:10]
most_desirables = scoring_result_ordered.head(n=10)

print(most_desirables['buildingqualitytypeid'].mean())
most_desirables

-0.7071146176209302


,parcelid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,transactiondate,score
141942,10862245,6.055848,0.339878,3.912069,5.338689,2.561337,0.988559,4.922572,2017-08-30,23.439196
60710,11650918,4.315227,-0.292207,3.434757,5.281846,2.402344,1.747990,5.050215,2016-01-11,22.524585
144297,11655139,3.444916,0.339878,4.309422,5.739236,2.787843,1.072940,5.269655,2017-04-07,22.284133
87193,11654953,3.444916,-0.292207,4.649799,5.673003,2.002286,1.326084,4.601712,2016-10-20,21.990007
115381,11641869,4.315227,0.339878,3.912069,5.878335,1.424656,1.410465,4.744048,2017-02-01,21.344922
61813,11670010,3.444916,-3.272181,3.685366,4.117994,0.474359,1.916753,4.409567,2016-12-29,21.321135
3971,11650903,5.185538,-0.292207,3.685366,4.768280,0.815932,1.452656,4.842481,2016-07-11,21.042459
79388,10883206,7.796470,-1.313993,3.154600,3.361436,0.881665,1.705800,2.331928,2016-09-27,20.545892
133332,11706270,11.277714,-1.313993,3.154600,2.637588,-0.587304,0.735415,1.464498,2017-07-20,19.996504
88506,10837531,7.796470,-1.313993,3.154600,2.883729,0.881665,1.621418,1.619915,2016-08-16,19.271790


In [15]:
# Get the ten least desirable houses in the dataset
least_desirables = scoring_result_ordered[-10:]
least_desirables = scoring_result_ordered.tail(n=10)

print(least_desirables['buildingqualitytypeid'].mean())
least_desirables

0.2959364818476111


,parcelid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,lotsizesquarefeet,yearbuilt,structuretaxvaluedollarcnt,transactiondate,score
88130,11807594,-1.776949,2.602381,-1.791531,-2.559796,-1.364967,-1.880403,-2.654792,2016-04-22,-14.630819
81433,12660907,-2.647260,-1.313993,0.821613,-15.013332,-0.096297,0.482272,0.374094,2016-03-25,-14.764918
72306,12231898,-1.776949,0.644194,-1.791531,-3.217609,-0.970576,-1.964784,-4.534240,2016-07-15,-14.899885
44273,11330994,-1.776949,0.644194,-1.791531,-3.205881,1.261461,-0.150588,-8.936016,2016-01-06,-15.243699
76819,11810855,-2.647260,-0.292207,0.199127,-5.172037,-0.526313,-1.964784,-6.073198,2016-10-07,-15.892259
41121,12059289,-1.776949,0.644194,-1.791531,-3.114202,-1.313725,-1.964784,-5.868466,2016-06-20,-16.473853
89349,11524235,-2.647260,-0.292207,0.199127,-8.285421,1.052183,1.537037,-8.887253,2016-02-26,-16.739380
157491,12664381,-2.647260,0.339878,0.206507,-5.673180,0.063147,-0.825638,-8.198329,2017-02-10,-17.414630
3631,10979838,-2.647260,0.644194,-1.791531,-2.761598,-1.231372,-1.880403,-6.693889,2016-03-08,-17.650248
90781,11589514,-1.776949,-0.661264,-1.791531,-2.310925,-1.363869,-2.682025,-8.448588,2017-03-13,-17.712624


# Task 2

# Task 3